<a href="https://colab.research.google.com/github/mkorob/DementiaDataSynthEval/blob/main/SyntheticDataGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Synthetic Data Generation

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [3]:
with open('OpenAI_key_personal.txt') as f:
    api_key = f.readlines()[0]

In [4]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [5]:
import pandas as pd

## 1. Define Topics and Data

In [6]:
df_final = pd.read_csv("/content/final_data_texts_w_features_v2.csv")

In [7]:
new_essay_topics =  ["A Day in My Life: Describe a typical day, from morning routines to evening wind-down.",
	"Travel Diary: Share experiences and photos from a recent trip or vacation.",
	"Book Review: Write about a book you recently read and what you thought of it.",
	"Cooking Adventure: Post a recipe you tried and how it turned out.",
	"Fitness Journey: Document your progress in a new workout regimen or fitness goal.",
	"Weekend Getaway: Share highlights and photos from a spontaneous weekend trip.",
	"Favorite Hobbies: Talk about your favorite hobbies and why you enjoy them.",
	"Personal Growth: Reflect on personal growth or a recent life lesson.",
	"Family Time: Describe a memorable family event or tradition.",
	"Movie Night: Review a movie you watched and your thoughts on it.",
	"Gardening Tips: Share tips and photos from your garden or plant collection.",
	"DIY Project: Document a DIY project you completed, including steps and photos.",
	"Pet Stories: Share cute or funny stories about your pets.",
	"Friendship Moments: Write about a recent fun outing or experience with friends.",
	"Music Playlist: Create a playlist of your current favorite songs and why you love them.",
	"Career Insights: Discuss recent achievements or challenges at work.",
	"Local Adventure: Explore a new place in your city or town and write about it.",
	"Seasonal Activities: Describe seasonal activities or traditions you look forward to.",
	"Mindfulness Practice: Share your experiences with mindfulness or meditation.",
"Home Decor: Post about recent changes or updates to your home decor."]

### Define Training Sets

Training set is 10 rows of data from each author

In [8]:
import numpy as np

In [9]:
np.random.seed(42)  # For reproducibility
df_final["split"] = 'test'  # Default all to test

# Iterate over each author and assign 10 random rows to train
authors = df_final["ID"].unique()
for author in authors:
    author_rows = df_final[df_final["ID"] == author]
    train_indices = author_rows.sample(n=10, random_state=42).index
    df_final.loc[train_indices, "split"] = 'train'

In [10]:
df_final.groupby(['ID', 'split']).count()

Unnamed: 0  group  filtered_words  determiner_noun_phrase_ratio  \
ID    split                                                                    
con1  test           10     10              10                            10   
      train          10     10              10                            10   
con2  test           10     10              10                            10   
      train          10     10              10                            10   
con4  test           10     10              10                            10   
      train          10     10              10                            10   
con5  test            5      5               5                             5   
      train          10     10              10                            10   
con6  test           10     10              10                            10   
      train          10     10              10                            10   
dem1  test           14     14              14                            14   
      train          10     10              10                            10   
dem10 test           10     10              10                            10   
      train          10     10              10                            10   
dem11 test           10     10              10                            10   
      train          10     10              10                            10   
dem3  test           10     10              10                            10   
      train          10     10              10                            10   
dem6  test           11     11              11                            11   
      train          10     10              10                            10   

             past_tense_verbs  word_count_snip  vp_np_ratio  \
ID    split                                                   
con1  test                 10               10           10   
      train                10               10           10   
con2  test                 10               10           10   
      train                10               10           10   
con4  test                 10               10           10   
      train                10               10           10   
con5  test                  5                5            5   
      train                10               10           10   
con6  test                 10               10           10   
      train                10               10           10   
dem1  test                 14               14           14   
      train                10               10           10   
dem10 test                 10               10           10   
      train                10               10           10   
dem11 test                 10               10           10   
      train                10               10           10   
dem3  test                 10               10           10   
      train                10               10           10   
dem6  test                 11               11           11   
      train                10               10           10   

             coordinate_phrases_ratio  subordinate_ratio  avg_word_length  \
ID    split                                                                 
con1  test                         10                 10               10   
      train                        10                 10               10   
con2  test                         10                 10               10   
      train                        10                 10               10   
con4  test                         10                 10               10   
      train                        10                 10               10   
con5  test                          5                  5                5   
      train                        10                 10               10   
con6  test                         10                 10               10   
      train       

## Run for each author

In [13]:
def prompt_on_topic_wo_bias(author, seed, essay_topic, use_train):

  prompt = """Your task is to generate podcast intros in the style of the author. Below are ten examples of the author's entries. Analyze these examples to understand the style, tone, and structure used. Then, create a new podcast transcript that mimics this style.

  Examples:"""
  if use_train:
    print("Using train subset for prompt generation")
    samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = seed)
  else:
    print(f"Generating samples with seed {seed}")
    samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  example_no = 1
  for sample in samples:
    prompt+= f"""

    Example {example_no}:
    {sample}"""
    example_no +=1

  prompt += f"\n\nWrite a blog entry on {essay_topic}"

  return prompt

In [ ]:
# def prompt_on_topic_w_bias(author, seed, essay_topic, use_train):

#   prompt = """Your task is to write a blog entry in the writing style of the author with dementia. Below are ten examples of the author's entries. Analyze these examples to understand the style, tone, and structure used. Then, create a new blog entry that mimics this style.

#   Examples:"""

#   if use_train:
#     print("Using train subset for prompt generation")
#     samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = seed)
#   else:
#     print(f"Generating samples with seed {seed}")
#     samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
#   example_no = 1
#   for sample in samples:
#     prompt+= f"""

#     Example {example_no}:
#     {sample}"""
#     example_no +=1

#   prompt += f"\n\nWrite a blog entry on {essay_topic}"

#   return prompt

In [12]:
def get_gpt3_predictions(prompts, model='gpt-3.5-turbo'):
    """
    Generate predictions for a list of prompts using GPT-3.5.

    Args:
    prompts (list of str): List of prompts to generate predictions for.
    model (str): The model to use for predictions (default: 'gpt-3.5-turbo').

    Returns:
    list of str: List of predictions generated by GPT-3.5.
    """
    predictions = []

    for prompt in tqdm.tqdm(prompts):
        response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo-0125",
    temperature = 0,
)
        prediction = response.choices[0].message.content.strip()
        predictions.append(prediction)

    return predictions

In [14]:
import tqdm

In [15]:
authors = df_final['ID'].unique()

In [16]:
authors_out = []
prompts_out = []
predictions_out = []

In [18]:
authors_out_train = []
prompts_out_train = []
predictions_out_train = []

In [19]:
#Version 1 - running predictions on 10-row "train set"
seed = 42
for author in df_final['ID'].unique():
  print(f"parsing author {author}")
  #samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = 42)
  #samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  prompts_first_speaker = [prompt_on_topic_wo_bias(author, seed, topic, use_train = True) for topic in new_essay_topics]
  predictions = get_gpt3_predictions(prompts_first_speaker)
  authors_out_train += [author]*len(predictions)
  prompts_out_train += prompts_first_speaker
  predictions_out_train += predictions

parsing author dem1
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:46<00:00,  5.31s/it]


parsing author dem3
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:42<00:00,  5.10s/it]


parsing author dem6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


...

100%|██████████| 20/20 [01:40<00:00,  5.01s/it]


parsing author con5
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:47<00:00,  5.35s/it]


parsing author con6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [01:49<00:00,  5.46s/it]


In [ ]:
#Version 2 - running predictions on constantly shuffled 10 rows
for author in df_final['ID'].unique():
  print(f"parsing author {author}")
  seed = 42
  #samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = 42)
  #samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  prompts_first_speaker = [prompt_on_topic_wo_bias(author, seed + i, topic, use_train = False) for i, topic in enumerate(new_essay_topics)]
  predictions = get_gpt3_predictions(prompts_first_speaker)
  authors_out += [author]*len(predictions)
  prompts_out += prompts_first_speaker
  predictions_out += predictions

parsing author dem1


100%|██████████| 20/20 [01:52<00:00,  5.61s/it]


parsing author dem3


100%|██████████| 20/20 [01:59<00:00,  5.96s/it]


parsing author dem6


...

100%|██████████| 20/20 [02:06<00:00,  6.34s/it]


parsing author con5


100%|██████████| 20/20 [02:13<00:00,  6.68s/it]


parsing author con6


100%|██████████| 20/20 [01:53<00:00,  5.66s/it]


In [ ]:
unbiased_predictions = pd.DataFrame(data= {'ID': authors_out, 'prompt': prompts_out, 'prediction': predictions_out})

In [ ]:
unbiased_predictions

,ID,prompt,prediction
0,dem1,Your task is to write a blog entry in the writ...,"Good morning, everyone! It's Agnes from Coatbr..."
1,dem1,Your task is to write a blog entry in the writ...,"Good morning, everyone! It's Agnes from Coatbr..."
2,dem1,Your task is to write a blog entry in the writ...,"Good morning, everyone. It's Agnes from Coatbr..."
3,dem1,Your task is to write a blog entry in the writ...,"Good morning, everyone! It's Agnes from Coatbr..."
4,dem1,Your task is to write a blog entry in the writ...,"Good morning everybody, it's Agnes from Coatbr..."
...,...,...,...
195,con6,Your task is to write a blog entry in the writ...,"Welcome to The Overwhelmed Brain, where we div..."
196,con6,Your task is to write a blog entry in the writ...,"Welcome to The Overwhelmed Brain, where we div..."
197,con6,Your task is to write a blog entry in the writ...,"As the seasons change, so do the activities an..."
198,con6,Your task is to write a blog entry in the writ...,"Welcome to The Overwhelmed Brain, where we div..."


In [20]:
unbiased_predictions_train = pd.DataFrame(data= {'ID': authors_out_train, 'prompt': prompts_out_train, 'prediction': predictions_out_train})

In [ ]:
unbiased_predictions.to_csv("generated_unbiased_predictions_shuffled.csv")

In [21]:
unbiased_predictions_train.to_csv("generated_unbiased_predictions_unshuffled_v2.csv")

In [ ]:
authors_out_biased = []
prompts_out_biased = []
predictions_out_biased = []

In [ ]:
authors_out_biased_train = []
prompts_out_biased_train = []
predictions_out_biased_train = []

In [ ]:
#Version 1 - running predictions on 10-row "train set"
seed = 42
for author in df_final['ID'].unique():
  print(f"parsing author {author}")
  prompts_first_speaker = [prompt_on_topic_w_bias(author, seed, topic, use_train = True) for topic in new_essay_topics]
  predictions = get_gpt3_predictions(prompts_first_speaker)
  authors_out_biased_train += [author]*len(predictions)
  prompts_out_biased_train += prompts_first_speaker
  predictions_out_biased_train += predictions

parsing author dem1
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [02:14<00:00,  6.71s/it]


parsing author dem3
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [02:07<00:00,  6.39s/it]


parsing author dem6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


...

100%|██████████| 20/20 [02:16<00:00,  6.82s/it]


parsing author con5
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [02:19<00:00,  7.00s/it]


parsing author con6
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation
Using train subset for prompt generation


100%|██████████| 20/20 [02:35<00:00,  7.80s/it]


In [ ]:
#Versoin 2 - running predictions on a reshuffled 10-row set
for author in df_final['ID'].unique():
  print(f"parsing author {author}")
  seed = 42
  #samples = df_final.loc[(df_final['ID'] == author) & (df_final['split'] == "train"), 'filtered_words'].sample(frac = 1, random_state = 42)
  #samples = df_final.loc[(df_final['ID'] == author), 'filtered_words'].sample(n=10, random_state = seed)
  prompts_first_speaker = [prompt_on_topic_w_bias(author, seed + i, topic, ues_train = False) for i, topic in enumerate(new_essay_topics)]
  predictions = get_gpt3_predictions(prompts_first_speaker)
  authors_out_biased += [author]*len(predictions)
  prompts_out_biased += prompts_first_speaker
  predictions_out_biased += predictions

parsing author dem1


100%|██████████| 20/20 [01:44<00:00,  5.21s/it]


parsing author dem3


100%|██████████| 20/20 [01:37<00:00,  4.86s/it]


parsing author dem6


...

100%|██████████| 20/20 [01:53<00:00,  5.66s/it]


parsing author con5


100%|██████████| 20/20 [01:54<00:00,  5.72s/it]


parsing author con6


100%|██████████| 20/20 [01:53<00:00,  5.70s/it]


In [ ]:
biased_predictions = pd.DataFrame(data= {'ID': authors_out_biased, 'prompt': prompts_out_biased, 'prediction': predictions_out_biased})

In [ ]:
biased_predictions_train = pd.DataFrame(data= {'ID': authors_out_biased_train, 'prompt': prompts_out_biased_train, 'prediction': predictions_out_biased_train})

In [ ]:
biased_predictions.loc[0, 'prompt']

"Your task is to write a blog entry in the writing style of the author with dementia. Below are ten examples of the author's entries. Analyze these examples to understand the style, tone, and structure used. Then, create a new blog entry that mimics this style.\n\n  Examples:\n\n    Example 1:\n    Good morning everybody this is Agnes from Coatbridge calling.A very wet damp grey Coatbridge , and I know all my friends in England and other parts of Scotland are having nice summery weather . What I wanted to say today was not to give you a weather report but to say I think I ` m getting my MoJo back ! I have only got one more day of two weeks antibiotics and steroids and I think I ` m beginning to feel I ` m making headway.I know when I ` m making headway because I start to plan and prepare and what I wanted to say was this wasnâ€™t the help from the medical profession because I have not seen them , havenâ€™t spoken to any , no that ` s not true , I havenâ€™t spoken to my GP . I was triag

In [ ]:
unbiased_predictions.loc[0, 'prompt']

"Your task is to write a blog entry in the writing style of the author. Below are ten examples of the author's entries. Analyze these examples to understand the style, tone, and structure used. Then, create a new blog entry that mimics this style.\n\n  Examples:\n\n    Example 1:\n    Good morning everybody this is Agnes from Coatbridge calling.A very wet damp grey Coatbridge , and I know all my friends in England and other parts of Scotland are having nice summery weather . What I wanted to say today was not to give you a weather report but to say I think I ` m getting my MoJo back ! I have only got one more day of two weeks antibiotics and steroids and I think I ` m beginning to feel I ` m making headway.I know when I ` m making headway because I start to plan and prepare and what I wanted to say was this wasnâ€™t the help from the medical profession because I have not seen them , havenâ€™t spoken to any , no that ` s not true , I havenâ€™t spoken to my GP . I was triaged with respir

In [ ]:
biased_predictions.to_csv("generated_biased_predictions_shuffled.csv")

In [ ]:
biased_predictions_train.to_csv("generated_biased_predictions_unshuffled.csv")